# Notebook de pruebas para llegar al algoritmo final de recomendación implementado en la WEB

In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from joblib import dump, load
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
productos = pd.read_csv('../data/productos.csv', index_col=0) 

In [12]:
productos

,ARTICULO,DESCRIPCION
0,002810071,GRIFO LLENAVASOS PARA FUENTES DE AGUA REF. 100...
1,0044152426461,TUBO DE ENLACE KOMBIFIX CON TUERCAS REF. 152.4...
2,0044240705001,FLOTADOR UNIFILL REF. 240.705.00.1 PARA CISTER...
3,0060PDSGPZ10070BL,Plato de ducha de carga mineral textura pizarr...
4,00R00601,VERTEDERO MODELO GARDA A371055000 ROCA
...,...,...
19846,0901304498,Manguito portabrida 315 pvc presión
19847,0901304500,Brida loca presión 315 de 12 agujeros
19848,090658707,"SKIMMER 17,5L NORM para piscina de hormigón b..."
19849,1012121NE,Fregadero sobre encimera CORAL de 1 cubeta y e...


In [13]:
cestas_su = pd.read_csv('../data/cestas_su.csv', index_col=0) 
cestas_su

,Cestas
1,014697462000 0438UAES
2,0274IGTH221222 077073154N 2001WPDIAZ
3,0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
4,07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL
7,0274AI20AM042040 0274IGCV45HM42
...,...
61387,0201CU2431210 0201CU2431512 0201CU2715 0422VI1...
61388,00R9822502100 00R9822502400 00R9V0028800R 0226...
61393,1600ES12 1600ES34
61394,0803B2003000 0804241250200 0816270250


# Eliminamos cestas duplicadas ya que con tener la información de uno de las cestas ya nos valdría. 

In [14]:
cestas_2 = cestas_su.drop_duplicates().reset_index(drop=True) 

In [15]:
cestas_2

,Cestas
0,014697462000 0438UAES
1,0274IGTH221222 077073154N 2001WPDIAZ
2,0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
3,07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL
4,0274AI20AM042040 0274IGCV45HM42
...,...
59315,0201CU2431210 0201CU2431512 0201CU2715 0422VI1...
59316,00R9822502100 00R9822502400 00R9V0028800R 0226...
59317,1600ES12 1600ES34
59318,0803B2003000 0804241250200 0816270250


In [16]:
cestas_2.to_csv('../data/cestas_final.csv')

# Las cestas que tienen solo un artículo han sido eliminadas

### Entrenamos el count

In [17]:
count_vectorizer = CountVectorizer()

In [18]:
count_vectorizer.fit(cestas_2['Cestas'])

CountVectorizer()

### Transformamos las cestas en matriz según el conteo de artículos en cada una

In [19]:
# Calcular la matriz de conteo
count_matrix_su_2 = count_vectorizer.transform(cestas_2['Cestas'])

### Normalizamos la matriz

In [20]:
# Normalizar la matriz de conteo por la suma total de términos en cada documento (aplicar TF)
tf_matrix_su = normalize(count_matrix_su_2, norm='l1')  # 'l1' normaliza por la suma de la fila

In [21]:
tf_matrix_su

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 686934 stored elements and shape (59320, 18840)>

### Guardamos el modelo y la matrix

In [22]:
# Guardar la matriz de conteo y el modelo en la carpeta 'models'
dump(count_vectorizer, '../models/count_vectorizer_2.joblib')
dump(tf_matrix_su, '../models/count_matrix_2.joblib')

['../models/count_matrix_2.joblib']

### Definimos la función que comparará una cesta actual con el histórico de cestas para ofrecer recomendaciones

In [34]:
def recomienda_tf(new_basket, cestas, productos):  # MEtemos como variable tambn productos por si se añadieran nuevos
    # Cargar la matriz TF y el modelo
    tf_matrix = load('../models/count_matrix_2.joblib')
                      
    count = load('../models/count_vectorizer_2.joblib')
    # Convertir la nueva cesta en formato TF (Term Frequency)
    new_basket_str = ' '.join(new_basket)
    new_basket_vector = count.transform([new_basket_str])
    new_basket_tf = normalize(new_basket_vector, norm='l1')  # Normalizamos la matriz count de la cesta actual
    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_tf, tf_matrix)
    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-4:]  # Las 4 más similares
    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0
    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score  # Suma de las similitudes
        products = cestas.iloc[idx]['Cestas'].split()
        # Usar un conjunto para evitar contar productos múltiples veces en la misma cesta
        unique_products = set(products)  # Usar un conjunto para obtener productos únicos
        # Con esto evitamos que la importancia crezca por las unidades
        for product in unique_products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score
                # Almacena el conteo de la relevancia de cada producto basado en cuántas veces aparece en las cestas similares, ponderado por la similitud de cada cesta.
    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")
     
    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación
    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, score in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'RELEVANCIA': score
            })
    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df.head(5)

# Pruebas

In [24]:
# Cestas para probar
new_basket = ['08049011087', '08049011087','0803B1103000', '0803B1103000']
new_basket_2 = ['00R1BTGCBL5','00R1BTGBL9']
new_basket_3 = ['0113LU01', '0113LU12']

In [35]:
recomendacion = recomienda_tf(new_basket_3, cestas_2, productos)
recomendacion

,ARTICULO,DESCRIPCION,RELEVANCIA
0,0113LU31,MONOMANDO DE DUCHA LUCA CROMADO FELIU BOET 14424,0.492941
1,0770BR0600CS,BARRA RECTA DE 600 mm INOX SAT 73233NP MEDICLI...,0.260206
2,00R9AH0003600R,MECANISMO DE DOBLE DESCARGA PARA CISTERNA DE W...,0.260206
3,0113LU21,MONOMANDO DE BAÑO DUCHA LUCA CROMADO FELIU BOE...,0.260206
4,1600ESMH1P,LLAVE DE PASO ESFERA MH DE 1 ECONOMICA PALOMILLA,0.246853


# Pruebas para función que nos permita retroalimentar nuestro algoritmo con nuevas cestas

### Definimos una función que tomará el csv de históricos de cestas y una cesta nueva. Si esta nueva cesta no está en el histórico, entonces se añadirá. Además, count se volverá a entrenar con este nuevo csv calculando de nuevo la matriz con esta nueva instancia. La idea es que se sobreescriban los archivos. Ahora mismo se guardan en uno nuevo denminado pruebas y los modelos y matriz final

In [26]:
# Función con la que podremos añadir nuevas cestas a nuestro histórico
# Cestas es el csv al que le añadimos la cesta_nueva
# Tener en cuenta que cesta_nueva es una lista
def retroalimentacion(cestas, cesta_nueva):
    # Pasamos de lista a cadena de texto
    cesta_unida = ' '.join(cesta_nueva)
    # Añadimos la cesta nueva al histórico de cestas. Primero comprobamos si la cesta nueva ya está
    if not cestas['Cestas'].isin([cesta_unida]).any():
        # Añadir la nueva cesta si no existe
        cestas.loc[len(cestas)] = cesta_unida
        print("Cesta añadida.")
        # Reescribimos la nueva cesta
        cestas.to_csv('../data/cestas_prueba.csv')
    else:
        print("La cesta ya existe en el DataFrame.")
    
    # Vectorizamos de nuevo el df de cestas y guardamos matriz y modelo
    count_vectorizer = CountVectorizer()

    count_vectorizer.fit(cestas['Cestas'])

    count_matrix = count_vectorizer.transform(cestas['Cestas'])

    # Normalizar la matriz de conteo por la suma total de términos en cada documento (aplicar TF)
    tf_matrix = normalize(count_matrix, norm='l1')  # 'l1' normaliza por la suma de la fila

    # Guardar la matriz de conteo y el modelo en la carpeta 'models'
    dump(count_vectorizer, '../models/count_vectorizer_final.joblib')
    dump(tf_matrix, '../models/tf_matrix_final.joblib')
    

    return None


# Pruebas

In [27]:
# Hago una copia de cestas_2
cestas_pruebas = cestas_2.copy() # Ejecutar esta celda cuando quiera resetear la cesta de pruebas a la original

In [28]:
cestas_pruebas

,Cestas
0,014697462000 0438UAES
1,0274IGTH221222 077073154N 2001WPDIAZ
2,0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
3,07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL
4,0274AI20AM042040 0274IGCV45HM42
...,...
59315,0201CU2431210 0201CU2431512 0201CU2715 0422VI1...
59316,00R9822502100 00R9822502400 00R9V0028800R 0226...
59317,1600ES12 1600ES34
59318,0803B2003000 0804241250200 0816270250


In [29]:
# Cestas nuevas para añadir.
cesta_nueva = ['0113LU01', '0113LU12']
cesta_nueva_2 = ['00R1BTGCBL5','00R1BTGBL9']

In [30]:
retroalimentacion(cestas_pruebas, cesta_nueva_2 )

Cesta añadida.


In [31]:
# Cargamos el csv con las cestas nuevas para comrprobar que se ha añadido la nueva
cesta_actualizada = pd.read_csv('../data/cestas_prueba.csv', index_col=0) 

In [32]:
cesta_actualizada.tail()

,Cestas
59316,00R9822502100 00R9822502400 00R9V0028800R 0226...
59317,1600ES12 1600ES34
59318,0803B2003000 0804241250200 0816270250
59319,0660046 069551010
59320,00R1BTGCBL5 00R1BTGBL9
